# 📓 The GenAI Revolution Cookbook

**Title:** GPT-4o transcription: How to Preserve Word Document Layouts in Python

**Description:** Preserve Word layouts, tables, and media with Python GPT-4o hybrid workflow; combine text extraction and page images for accurate transcription.

**📖 Read the full article:** [GPT-4o transcription: How to Preserve Word Document Layouts in Python](https://blog.thegenairevolution.com/article/gpt-4o-transcription-how-to-preserve-word-document-layouts-in-python)

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Word documents are everywhere. Legal contracts, business reports, academic papers, you name it. But here's the thing: when you try to parse them programmatically, they're a nightmare. Tables, images, annotations, styled text, all of that can make your standard extraction techniques completely miss the point. Or worse, they'll grab the content but lose all the context that makes it meaningful. If you're building an app that needs to actually understand these documents, not just read them, you can't afford to lose that structure. And if you need to pull structured data out of these documents, not just transcribe them, you might want to check out our guide on [extracting structured data from complex documents using GPT\-4o Vision](/article/invoice-data-extraction-how-to-get-json-via-gpt-4o-vision-3).

Sure, you could use python\-docx to extract the text. But then you're ignoring the layout and all the embedded media. You could convert everything to plain text and handle images separately, but good luck figuring out where those images actually belonged in the document. There's got to be a better way, right?

## The Hybrid Strategy: Best of Both Worlds

So GPT\-4o has this multi\-modal capability where you can send both images and text in one request. This is actually perfect for transcribing complex documents, especially when the layout and visuals actually matter to understanding the content.

The strategy I've been using is pretty straightforward:

1. First, convert the Word document to PDF using LibreOffice in headless mode
2. Convert that PDF to images, one image per page
3. Extract the raw text from each page (not using OCR, just pulling the text)
4. Send both the image and the text to GPT\-4o for each page


What this does is let GPT\-4o read the document the way a human would. It sees both the visual layout and has the underlying text available at the same time.

Let me walk you through how to actually implement this in Python.

## Setup

You'll need these libraries installed:

In [ ]:
pip install pdf2image openai python-docx PyMuPDF

Also, make sure LibreOffice is installed on your system and you can run it from the command line with the libreoffice command.

In [ ]:
sudo apt install libreoffice
sudo apt install poppler-utils

Load your API key securely:

In [ ]:
# Import the necessary Python libraries
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

# Load the OPENAI_API_KEY from local .env file
_ = load_dotenv(find_dotenv())

## Step 1: Convert Word to PDF using LibreOffice

In [ ]:
import subprocess
from pathlib import Path

def convert_docx_to_pdf(input_path):
    input_path = Path(input_path)
    output_dir = input_path.parent
    subprocess.run([
        "libreoffice", "--headless", "--convert-to", "pdf", str(input_path), "--outdir", str(output_dir)
    ], check=True)
    return output_dir / (input_path.stem + ".pdf")

## Step 2: Convert PDF Pages to Images

In [ ]:
from pdf2image import convert_from_path

def convert_pdf_to_images(pdf_path, dpi=300):
    images = convert_from_path(pdf_path, dpi=dpi)
    return images

## Step 3: Extract Text from Each Page (No OCR)

In [ ]:
import fitz  # PyMuPDF

def extract_text_per_page(pdf_path):
    doc = fitz.open(pdf_path)
    return [page.get_text() for page in doc]

## Step 4: Send Pages to GPT\-4o (Text \+ Image)

In [ ]:
import openai
import base64
import io

def encode_image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_b64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/png;base64,{img_b64}"

def transcribe_page_with_gpt4o(image, text):
    prompt_instruction = (
        "You are a transcription assistant. Carefully transcribe the content of this page. "
        "Use both the provided image and raw text. Focus on preserving the structure, layout, and accuracy. "
        "Do not ignore tables, headings, or embedded content."
    )

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_instruction},
                {"type": "text", "text": text},
                {"type": "image_url", "image_url": {"url": encode_image(image)}}
            ]
        }]
    )
    return response["choices"][0]["message"]["content"]

If you want to make the transcription quality even better, you might want to look into [in\-context learning techniques to improve LLM transcription quality](/article/the-magic-of-in-context-learning-teach-your-llm-on-the-fly-3). These techniques can really help guide the model's outputs based on what you specifically need.

## Putting it all together

In [ ]:
def transcribe_docx(docx_path):
    pdf_path = convert_docx_to_pdf(docx_path)
    images = convert_pdf_to_images(pdf_path)
    texts = extract_text_per_page(pdf_path)

    transcripts = []
    for image, text in zip(images, texts):
        transcription = transcribe_page_with_gpt4o(image, text)
        transcripts.append(transcription)
    return transcripts

## Testing it out

In [ ]:
transcripts = transcribe_docx("files/Acme_Strategy_2025_docx.docx")
for i, page in enumerate(transcripts, 1):
    print(f"--- Page {i} ---\n{page}\n")

<img src='https://thegenairevolution.com/wp-content/uploads/2025/05/Word-Documen%E2%80%A6-3-JupyterLab-05-12-2025_08_13_PMc-792x1024.png' alt='' title='' width='792' height='1024' />

## Why This Approach Works

The thing about this hybrid method is that you don't lose the important layout context. Figure labels, column structures, where headers sit relative to tables, all of that stays intact. This is especially valuable when you're dealing with documents where the visual arrangement actually carries meaning. A pure text extractor just can't capture that.

When you pass both the raw text and the page image to GPT\-4o, you're giving it the complete picture. Literally. The model can then reason across both the text and the layout to give you cleaner transcriptions that actually understand the context.

If you're working with multiple documents and need to retrieve information from them, our guide on [building multi\-document agents for retrieval and summarization](/article/multi-document-agent-with-llamaindex-the-ultimate-guide-2025-2) has some advanced strategies that work really well with this hybrid approach.

## Other alternatives

When you're converting Word documents to PDF on Linux with Python, the LibreOffice CLI approach gives you high fidelity without much setup hassle. Here's how it compares to other popular methods:

Method Fidelity Dependencies Complexity LibreOffice CLI High LibreOffice Low unoconv High LibreOffice \+ unoconv Low to Medium UNO Automation High LibreOffice listener \+ uno Python bindings High Pandoc Medium Pandoc \+ TeX engine (like TeX Live) Low HTML to PDF (mammoth\+pdfkit) Medium mammoth \+ wkhtmltopdf/WeasyPrint Medium Aspose.Words (commercial) Very High Aspose.Words license Low Cloud API (Graph/Drive/etc.) High Internet access \+ API credentials Medium

Once you've got your documents accurately transcribed, the next step is usually optimizing how you retrieve information from them. You might find our article on [improving retrieval accuracy with cross\-encoder reranking](/article/cross-encoder-reranking-the-low-cost-fix-for-rag-misses) helpful for making sure your system surfaces the most relevant information.

## Conclusion

Word documents are notoriously difficult to process because of their complex internal structure. But this hybrid method gives you a clean, accurate way to transcribe them while preserving the layout, using GPT\-4o. No OCR necessary.

Whether you're building a search engine, a data extraction tool, or an archival system, this approach gives you high\-quality output without many compromises. And if you're thinking about integrating document understanding into conversational AI systems, take a look at our tutorial on [building a knowledge graph chatbot with Neo4j, Chainlit, and GPT\-4o](/article/how-to-build-a-knowledge-graph-chatbot-with-neo4j-chainlit-gpt-4o-3) for a practical example of how to apply this.